In [ ]:
# !ssh davandenakker@login.delftblue.tudelft.nl

#RNN seq2seq PROGRAM

## Install dependencies

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Adrian
[nltk_data]     Segura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Clone the KQA Pro repository

In [2]:
!git clone https://github.com/shijx12/KQAPro_Baselines.git

fatal: destination path 'KQAPro_Baselines' already exists and is not an empty directory.


In [2]:
cd KQAPro_Baselines

[WinError 2] The system cannot find the file specified: 'KQAPro_Baselines'
c:\Users\Adrian Segura\Desktop\TUDelft\Q4\KQAPro_Baselines


In [4]:
!pip install tqdm


[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import urllib.request
import zipfile
import os
from tqdm import tqdm, trange
import numpy as np
import pickle


In [7]:
pip install --force-reinstall certifi

  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2021.10.8
    Uninstalling certifi-2021.10.8:
      Successfully uninstalled certifi-2021.10.8

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


In [8]:


def download_and_unzip(url, path):
  zip_file_path = f"{path}.zip"
  urllib.request.urlretrieve(url, zip_file_path)
  # Path to extract the contents of the zip file
  # Extract the zip file
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(path)
  os.remove(zip_file_path)
  print("Finished downloading and extracting", path)

download_and_unzip(
    "https://cloud.tsinghua.edu.cn/f/04ce81541e704a648b03/?dl=1",
    "./dataset"
)
download_and_unzip(
    "http://nlp.stanford.edu/data/glove.840B.300d.zip",
    "./glove"
)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>

In [16]:
cd ..

c:\Users\Adrian Segura\Desktop\TUDelft\Q4\KQAPro_Baselines


In [17]:
chunk_size = 100000  # Number of entries per chunk
chunk_index = 0

res = {}
ctr = 0
for line in tqdm(open("./glove/glove.840B.300d.txt", encoding="latin-1")):
    word, *vec = line.split()
    try:
        vec = np.asarray(list(map(float, vec)))
        res[word] = vec
    except Exception as e:
        pass

    # Save the dictionary in chunks
    if len(res) >= chunk_size:
        with open(f"./glove/glove_{chunk_index}.pkl", 'wb') as f:
            pickle.dump(res, f)
        res = {}
        chunk_index += 1

# Save the remaining entries in the last chunk
if len(res) > 0:
    with open(f"./glove/glove_{chunk_index}.pkl", 'wb') as f:
        pickle.dump(res, f)


2196017it [03:42, 9852.15it/s] 


In [18]:
del res

In [19]:
merged_res = {}
for i in trange(chunk_index + 1):
    with open(f"./glove/glove_{i}.pkl", 'rb') as f:
        chunk_dict = pickle.load(f)
    merged_res.update(chunk_dict)

# Save the merged dictionary to a single file
with open("./glove/glove_merged.pkl", 'wb') as f:
    pickle.dump(merged_res, f)

100%|██████████| 22/22 [00:18<00:00,  1.17it/s]


In [20]:
# delete chunks
for i in range(chunk_index + 1):
    file_path = f"./glove/glove_{i}.pkl"
    if os.path.exists(file_path):
        os.remove(file_path)

In [21]:
file_path = "./glove/glove.840B.300d.txt"
line_count = 0

with open(file_path, 'r', encoding='latin-1') as file:
    for line in file:
        line_count += 1

print("Number of lines:", line_count)

KeyboardInterrupt: 

In [ ]:
ls ./glove/

glove.840B.300d.txt  glove.pkl


In [ ]:
!python -m utils.pickle_glove --input ./glove/glove.840B.300d.txt --output ./glove/glove.pkl >> ignore.txt

2196017it [03:52, 9442.29it/s]
/bin/bash: line 1: 13374 Killed                  python -m utils.pickle_glove --input ./glove/glove.840B.300d.txt --output ./glove/glove.pkl >> ignore.txt


In [24]:
conda install torch

Retrieving notices: ...working... done
Note: you may need to restart the kernel to use updated packages.



EnvironmentLocationNotFound: Not a conda environment: c:\Users\Adrian



In [25]:
!python -m Program.preprocess --input_dir ./dataset/KQAPro.IID/ --output_dir ./processed/

Load questions
Build question vocabulary
Dump vocab to ./processed/vocab.json
word_token_to_idx:80201
function_token_to_idx:31
answer_token_to_idx:81629
Encode train set
shape of questions, functions, func_depends, func_inputs, choices, answers:
(94376, 85)
(94376, 17)
(94376, 17, 2)
(94376, 17, 3)
(94376, 10)
(94376,)
Encode val set
shape of questions, functions, func_depends, func_inputs, choices, answers:
(11797, 61)
(11797, 16)
(11797, 16, 2)
(11797, 16, 3)
(11797, 10)
(11797,)
Encode test set
shape of questions, functions, func_depends, func_inputs, choices, answers:
(11797, 51)
(0,)
(0,)
(0,)
(11797, 10)
(0,)



100%|██████████| 94376/94376 [00:11<00:00, 8161.04it/s]

100%|██████████| 11797/11797 [00:01<00:00, 6591.89it/s]

100%|██████████| 11797/11797 [00:01<00:00, 9912.78it/s] 


In [27]:
mv ./dataset/KQAPro.IID/kb.json ./processed/

SyntaxError: invalid syntax (4175002378.py, line 1)

### train baselines

In [ ]:
# # mount glove
# from google.colab import drive
# drive.mount('/content/drive/TUDelft/Q4/')

MessageError: ignored

In [28]:
# fix kb key error KeyError: 'predicate'
import json
# Load the JSON file
with open("./processed/kb.json", "r") as file:
    kb_data = json.load(file)

# Print the first element
if len(kb_data) > 0:

    for ent_id in kb_data['entities']:
        for rel_info in kb_data['entities'][ent_id]['relations']:
            rel_info['predicate'] = None



    # Print the "instanceOf" pairs in the concepts dictionary
    print("InstanceOf pairs in the concepts dictionary:")
    for key, value in kb_data['concepts'].items():
        if "instanceOf" not in value:
          kb_data['concepts'][key]["instanceOf"] = []
else:
    print("The knowledge base is empty.")

with open("./processed/kb.json", "w") as file:
    json.dump(kb_data, file)

InstanceOf pairs in the concepts dictionary:


In [4]:
import torch
c = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
c

'cuda'

In [ ]:
from Program import train

In [25]:
!python -m Program.train --input_dir ./processed/ --save_dir ./checkpoint --glove_pt ./glove/glove_merged.pkl --num_epoch 1 --batch_size 64

#vocab of word: 80201
#vocab of answer: 80257
load kb
torch.Size([64, 61, 768])
shape sorted.. tensor([30, 30, 28, 28, 28, 26, 26, 25, 24, 23, 23, 22, 22, 22, 22, 21, 21, 20,
        20, 20, 20, 20, 19, 19, 19, 18, 18, 17, 17, 17, 17, 16, 16, 16, 16, 16,
        15, 14, 14, 14, 13, 13, 13, 13, 13, 12, 12, 11, 11, 11, 11, 11, 11, 10,
        10, 10, 10,  9,  9,  8,  8,  8,  7,  6], device='cuda:0')
torch.Size([64, 61, 768])
shape sorted.. tensor([41, 35, 30, 30, 28, 25, 23, 22, 22, 22, 20, 20, 19, 19, 18, 18, 18, 17,
        17, 17, 16, 16, 15, 15, 15, 15, 15, 15, 15, 14, 14, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 12, 12, 12, 11, 11, 11, 11, 10, 10, 10,  9,  9,  9,  9,
         9,  9,  9,  9,  8,  8,  8,  7,  7,  7], device='cuda:0')
torch.Size([64, 61, 768])
shape sorted.. tensor([34, 31, 31, 28, 28, 28, 27, 26, 26, 25, 25, 24, 23, 23, 22, 22, 22, 21,
        21, 21, 20, 20, 20, 19, 19, 19, 18, 18, 18, 18, 17, 17, 17, 17, 17, 16,
        16, 16, 16, 16, 16, 16, 16, 15, 14, 14, 14,

2023-06-17 03:13:02,883 INFO     input_dir:./processed/
2023-06-17 03:13:02,883 INFO     save_dir:./checkpoint
2023-06-17 03:13:02,883 INFO     glove_pt:./glove/glove_merged.pkl
2023-06-17 03:13:02,883 INFO     ckpt:None
2023-06-17 03:13:02,883 INFO     lr:0.001
2023-06-17 03:13:02,883 INFO     weight_decay:1e-05
2023-06-17 03:13:02,883 INFO     num_epoch:1
2023-06-17 03:13:02,884 INFO     batch_size:64
2023-06-17 03:13:02,884 INFO     seed:666
2023-06-17 03:13:02,884 INFO     dim_word:300
2023-06-17 03:13:02,884 INFO     dim_hidden:1024
2023-06-17 03:13:02,937 INFO     Create train_loader and val_loader.........
2023-06-17 03:13:08,303 INFO     Create model.........
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a 

In [38]:
fo = open("./checkpoint/log.txt", "wb")

# Close opend file
fo.close()

In [ ]:
import pickle
with open("./glove/glove.pkl", 'rb') as f:
    d = pickle.load(f)


EOFError: ignored

# NSM
Here we attempt to train a NSM following the repository: https://github.com/RichardHGL/WSDM2021_NSM

## Loading pre-processed data
The authors provide files for the WebQSP dataset, already pre-processed to be used to train the Neural State Machine depicted in their paper: https://arxiv.org/pdf/2101.03737.pdf

As mentioned in the repository